# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

\**ANSWERS FOR PART 1 IN THIS CELL*\*

# Holdout, Cross-Validation and most commonly used in Deep NNs

Two practical methods used to estimate a model's performance are Holdout and Cross Validation. The idea behind both is that the training data can provide valuable insights over its distribution that we can use for both learning the right parameters and assessing our general model performance. 

In the case of Holdout, the training set is split into training and test data, with a usual ratio of 80:20. Once the learning algorithm computes its prediction parameters, we assess how well this model behaves on the test, unseen data - it is important that these two sets are disjoint, as we do not wish to train our model using testing datapoints. Note that the training set is further split into two: the actual training set and the validation set. The validation set is used to tune the hyperparameters, model parameters that the learning algorithm does not seek as that is usually too expensive to compute. However, the validation set only has a role during training for hyperparameters optimisation and thus underestimates the actual generalization error - that is the role of the testing set, where we evaluate how our model generalizes once the model parameters are tuned.

However, the Holdout method becomes problematic when the available dataset is small. Usually, a small dataset is unlikely to represent the actual underlying distribution of data and, moreover, splitting it and using the test data only after the training is done might lead to a poor model. The solution for a better mean test error when the dataset is too small comes with the price of a higher computational cost. The idea behind methods such as k-fold Cross Validation is randomly generating a number of k different partitions of our dataset. Usually, the whole dataset is partitioned into k disjoint sets and, at iteration i, the ith subset of data becomes test data, while the other k - 1 are used for training, as done in Holdout. We repeat the training on each of these k different splits and average the test error over each partition. This solution aims to improve on the uncertainty over the estimated average test error.

Deep Learning models are specifically efficient for problems where large datasets are collected. Thus, the Holdout method is more commonly used, as we tend to believe that large sets of data might represent the actual underlying distributions pretty well. Also, as the sets sizes increase, using Cross Validation might add expensive computations, slowing down the training process considerably, so Holdout remains the better solution for Deep Learning applications.

# Solutions to overfitting

L1, L2 Regularization - used to penalize big parameter values by adding a regularization term, i.e. from $\min L(\theta)$ to $\min L(\theta) + \beta R(\theta)$. From a statistical point of view, this is the same as adding a prior on the parameters. L2 ($|\theta|^2$) has the advantage of penalizing influential features less than the others, while the L1 ($|\theta|$) provides a sparse parameter space selection, i.e. lots of parameters will converge to 0.

Early Stop - the idea is to consider the number of training steps as another hyperparameter to be tuned. This technique tackles the problem of specializing the model too much on the training data by saving the parameters that obtain the lowest validation set error (and, hopefully, a good enough test error). During training, we store the parameters for every iteration that improves the error on the validation set and, when the training finishes, we return these parameters rather than those that were obtained last.

Dropout - zeroes the output of a node in the Neural Network setting with a certain probability. Hidden layers try to approximate the generator function and tend to fit a mapping that is maybe too specific, overfitting the data. By applying different network settings, i.e. by keeping a node with a certain probability (usually around 80%), the model becomes simpler and redundant features are less likely to be learnt by the network.

Data augmentation - depending on what the dataset actually contains, we might want to add more data for our model by applying some transformations on the already existing one. By training our model on more data, we hope to reduce the generalization error. This is specifically simple for classification problems - for a specific input $(x, y)$, we wish to apply a transformation function that keeps the label unchanged, i.e. $f(x) = x^{\prime}$ in order to obtain fake data as $(x^{\prime}, y)$. Object recognition is a good example where we could apply this technique, as we can apply geometric transformations on pixels (e.g. rotations, translations) while still preserving the actual objects encountered in a picture. Some intuition behind this is that our Network might now be less prone to specific, ungeneralizable features, such as some pixel in different pictures always having the same color.

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*

# 1. Sigmoid and tanh for hidden layers in Deep Fully Connected NNs

The sigmoid and tanh activation functions used to be pretty common as non-linear functions in fully-connected layers, however they lead to an issue known as the vanishing gradient problem. That is, the initialized weights are too big, these can saturate and entirely stop learning during the backward pass - the training loss will stop decreasing. The two functions saturate when the argument is very positive/negative, meaning that the functions become very flat and insensitive to small changes in the input. This happens as the derivative of sigmoid $z = \frac{1}{1 + e^{-x}}$ is $z (1 - z)$, while the derivative of $z = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ is $1 - z^2$. As $x$ becomes too large (in both negative and positive directions), the sigmoid function becomes either $0$ or $1$ (i.e., take the limit of $x$ to $+-\infty$), thus its derivative becomes $0$. Same happens for tanh, as $\lim z^2 = 1$ when $x$ goes to $+- \infty$. Thus, from this stage all the gradients become zero due to the chain rule. The gradient becomes null (vanishes), and the backward pass will keep the weights unchanged - the network stops learning.

# 2. Better as output unit activation functions

The sigmoid function can be used as an output activation function when the log-likelihood loss function is used. Using this combination is better than using sigmoid for hidden layers, as it ensures there is always a strong gradient whenever the model has the wrong answer. The gradient when using this combination is proportional to $(1 - \sigma (y_i \omega^T x_i))$ - saturation occurs only when the model already has the right answer - when $y_i = 1$ and $\omega^Tx_i$ is very positive, or $y_i = -1$ and $\omega^Tx_i$ is very negative. For the case when $\omega^T x_i$ has the wrong sign, the gradient is not shrinked at all and the algorithm corrects the weights quickly (see proof in the Deep Learning Book, page 183).

# 3. Zero-initialized weights

The problem with zero-initialized weights in Neural Networks settings is that the initial parameters will not break symmetry between different units. As explained in Goodfellow, page 301, "If two hidden units with the same activation function are connected to the same inputs, then these units must have different initial parameters". Having same parameters will result in  an exact similar behaviour for different units, making the algorithm deterministic in terms of what parameters will be learnt at the end. Thus, having each unit compute different functions requires a random initialization of the weights during the first training iteration.

This is not the case for logistic regression. First of all, the logistic regression is, by definition, only one function, so there are no other units that might emulate the same results. Also, in the case of logistic regression, zero-initialized weights ensure that the sigmoid values will stay within the linear area, thus ensuring a good propagation and a non-zero gradient. The gradient-based learning solution will eventually reach a local optimum.

## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *

Processing...
Done!


In [3]:
# Utility functions

def get_layer_params(in_features, out_features, weight_init_sd=1.0):
    nd = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([weight_init_sd]))
    params = (nd.sample(sample_shape=torch.Size([in_features, out_features]))).squeeze()
    params.requires_grad_()
    return params

def add_bias(inputs):
    return torch.cat([inputs, torch.ones([inputs.shape[0], 1])], dim=1)

In [4]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier():
    def __init__(self, weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.params = get_layer_params(785, 10, weight_init_sd=weight_init_sd)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
                
    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **        
        #######################################################################    
        h = inputs.view(-1, 784)
        h = add_bias(h)
        return F.log_softmax(h @ self.params, dim=1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.params]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(torch.sum(torch.abs(self.params), dim=1), dim=0)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(torch.sum(self.params ** 2, dim=1), dim=0)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [9]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
def run_with_model(lr=0.003, l1=0., l2=0.001, suppress_output=False, epochs=10, weight_init_sd=1.0):
    print("learning rate: ", lr, ", l2 coefficient: ", l2, " l1 coefficient: ", l1)
    model = MultinomialLogisticRegressionClassifier(weight_init_sd=weight_init_sd)
    res = run_experiment(
        model,
        optimizer=optim.Adam(model.parameters(), lr=lr),
        train_loader=train_loader_1,
        val_loader=val_loader_1,
        test_loader=test_loader_1,
        n_epochs=epochs,
        l1_penalty_coef=l1,
        l2_penalty_coef=l2,
        suppress_output=suppress_output
    )
    return res

# lr: 0.003, l2: 0.001, weight: 1.0, acc: 0.915
# Test set:	Average loss: 0.3061, Accuracy: 0.9160

lr = 0.00305
l1 = 0.0
l2 = 0.001
weight = 1.0
epochs = 10

res = run_with_model(lr=lr, l2=l2, l1=l1, epochs=epochs, weight_init_sd=weight)

learning rate:  0.00305 , l2 coefficient:  0.001  l1 coefficient:  0.0
Epoch 0: training...
Train set:	Average loss: 6.2320, Accuracy: 0.6592
Validation set:	Average loss: 2.1162, Accuracy: 0.8187

Epoch 1: training...
Train set:	Average loss: 1.5808, Accuracy: 0.8463
Validation set:	Average loss: 1.2596, Accuracy: 0.8650

Epoch 2: training...
Train set:	Average loss: 1.0272, Accuracy: 0.8686
Validation set:	Average loss: 0.8726, Accuracy: 0.8765

Epoch 3: training...
Train set:	Average loss: 0.7296, Accuracy: 0.8781
Validation set:	Average loss: 0.6847, Accuracy: 0.8725

Epoch 4: training...
Train set:	Average loss: 0.5596, Accuracy: 0.8850
Validation set:	Average loss: 0.5240, Accuracy: 0.8838

Epoch 5: training...
Train set:	Average loss: 0.4505, Accuracy: 0.8911
Validation set:	Average loss: 0.4714, Accuracy: 0.8892

Epoch 6: training...
Train set:	Average loss: 0.3859, Accuracy: 0.8989
Validation set:	Average loss: 0.4146, Accuracy: 0.8930

Epoch 7: training...
Train set:	Average 

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [48]:
# *CODE FOR PART 4.1 IN THIS CELL*
class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_1 = get_layer_params(785, 128, weight_init_sd=weight_init_sd)
        params_2 = get_layer_params(129, 64, weight_init_sd=weight_init_sd)
        params_3 = get_layer_params(65, 32, weight_init_sd=weight_init_sd)
        params_4 = get_layer_params(33, 10, weight_init_sd=weight_init_sd)        
                
        self.params = [params_1, params_2, params_3, params_4]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        h = inputs.view(-1, 784)
                
        for indx, param in enumerate(self.parameters()):
            if indx == len(self.parameters()) - 1:
                break
            h = add_bias(h)
            h = self.activation(h @ param)
        
        h = add_bias(h)
        return F.log_softmax(h @ self.parameters()[3], dim=1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return self.params
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        S = 0.
        
        for param in self.params:
            S += torch.sum(torch.sum(torch.abs(param), dim=1), dim=0)
        return S
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        S = 0.
        for param in self.params:
            S += torch.sum(torch.sum(param ** 2, dim=1), dim=0)
        return S    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

In [53]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

# lr = 0.0043
# l1 = 0.0
# l2 = 0.001
# weight = 1.0
# func = 'relu'
# Test set:	Average loss: 0.1473, Accuracy: 0.9572

lr = 0.0043
l1 = 0.0
l2 = 0.001
weight = 1.0
func = 'relu'
# Test set:	Average loss: 0.1191, Accuracy: 0.9637

model = MultilayerClassifier(activation_fun=func, weight_init_sd=weight)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), lr),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=l1,
    l2_penalty_coef=l2,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.2800, Accuracy: 0.9144
Validation set:	Average loss: 0.2051, Accuracy: 0.9382

Epoch 1: training...
Train set:	Average loss: 0.1815, Accuracy: 0.9457
Validation set:	Average loss: 0.1681, Accuracy: 0.9505

Epoch 2: training...
Train set:	Average loss: 0.1669, Accuracy: 0.9493
Validation set:	Average loss: 0.1588, Accuracy: 0.9498

Epoch 3: training...
Train set:	Average loss: 0.1587, Accuracy: 0.9520
Validation set:	Average loss: 0.1591, Accuracy: 0.9530

Epoch 4: training...
Train set:	Average loss: 0.1527, Accuracy: 0.9529
Validation set:	Average loss: 0.1584, Accuracy: 0.9538

Epoch 5: training...
Train set:	Average loss: 0.1486, Accuracy: 0.9549
Validation set:	Average loss: 0.1590, Accuracy: 0.9525

Epoch 6: training...
Train set:	Average loss: 0.1471, Accuracy: 0.9553
Validation set:	Average loss: 0.1595, Accuracy: 0.9535

Epoch 7: training...
Train set:	Average loss: 0.1480, Accuracy: 0.9557
Validation set:	Average loss: 0.1880, Ac

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*

An alternative to what we have used so far for weight initialization would be to use the He or Xavier initialization strategies. Instead of using the same weight variance for all parameters, we could make use of the layers' number of nodes to fill in the variance. The He initialization uses as variance $\frac{1}{2 \cdot n_{\text{in}}}$, while the Xavier initialization uses as variance $\frac{2}{n_{\text{in}} + n_{\text{out}}}$. Considering our setting, the He initialization could provide better results, as it is apparently better for ReLu settings.